In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier



import operator
import os


import scipy as sp
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing 

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

from collections import Counter
from sklearn.metrics import confusion_matrix,classification_report, f1_score
from sklearn.model_selection import cross_val_score
import warnings
import time

import matplotlib.pyplot as plt
from sklearn import svm

from sklearn.feature_selection import VarianceThreshold

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

from mlxtend.classifier import StackingClassifier

from sklearn.multiclass import OneVsRestClassifier
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek


import re
from time import time
import warnings
import joblib
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [3]:
data = pd.read_csv('/Volumes/MyDisk/Spring2020/Data Mining/deepsolar_tract.csv', encoding='latin1' )
data.head()

,Unnamed: 0,tile_count,solar_system_count,total_panel_area,fips,average_household_income,county,education_bachelor,education_college,education_doctoral,...,incentive_count_nonresidential,incentive_residential_state_level,incentive_nonresidential_state_level,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate
0,0,0.0,0.0,0.000000,27145011200,70352.789869,Stearns County,569,1690,13,...,39,11,13,34,0,0,25,12,0,9.46
1,1,25.0,21.0,1133.436461,27145011301,61727.085202,Stearns County,674,1434,108,...,39,11,13,34,0,0,25,12,0,9.46
2,2,3.0,3.0,64.505776,27145011302,71496.886583,Stearns County,854,1459,31,...,39,11,13,34,0,0,25,12,0,9.46
3,3,0.0,0.0,0.000000,27145011304,86840.152755,Stearns County,640,1116,68,...,39,11,13,34,0,0,25,12,0,9.46
4,4,5.0,5.0,164.583303,27145011400,89135.315597,Stearns County,654,1314,15,...,39,11,13,34,0,0,25,12,0,9.46


In [4]:
# Checking the dimension of the data - [rows, columns]

data.shape

(72537, 169)

In [646]:
# Looking into missing values

data.isnull().sum()

Unnamed: 0                                     0
tile_count                                     0
solar_system_count                             0
total_panel_area                               0
fips                                           0
average_household_income                     886
county                                         0
education_bachelor                             0
education_college                              0
education_doctoral                             0
education_high_school_graduate                 0
education_less_than_high_school                0
education_master                               0
education_population                           0
education_professional_school                  0
employed                                       0
gini_index                                   847
heating_fuel_coal_coke                         0
heating_fuel_electricity                       0
heating_fuel_fuel_oil_kerosene                 0
heating_fuel_gas    

#### Slicing to get State of California data only

In [5]:
# Slicing the dataset to reflect the State of California 

# And the checking the sliced data

cali_data = data[data['state']=='ca']
cali_data.shape

(8056, 169)

In [633]:
# Looking into the total number of missing values each variable has

cali_data.isnull().sum()

Unnamed: 0                                   0
tile_count                                   0
solar_system_count                           0
total_panel_area                             0
fips                                         0
average_household_income                    84
county                                       0
education_bachelor                           0
education_college                            0
education_doctoral                           0
education_high_school_graduate               0
education_less_than_high_school              0
education_master                             0
education_population                         0
education_professional_school                0
employed                                     0
gini_index                                  81
heating_fuel_coal_coke                       0
heating_fuel_electricity                     0
heating_fuel_fuel_oil_kerosene               0
heating_fuel_gas                             0
heating_fuel_

      ** In the following secton, Only State of California data is analysed.

## Census Tract Level Environmental and Socioeconomic Factors' Analysis


### 1. Statewide Incentive Parameters and Average Electricity Retail Rate Parameters

In [8]:
incentive_parameters = [

                        'incentive_count_residential',
                        'net_metering',
                        'feedin_tariff',
                        'cooperate_tax',
                        'property_tax',
                        'sales_tax',
                        'rebate',
                        'avg_electricity_retail_rate',
                        'electricity_price_residential',                                     

                       ]
len(incentive_parameters)

9

In [9]:
# Slicing the Incentive and Electricity retail rate variables and reviewing the first ten elements

cali_inc_data = cali_data[incentive_parameters]
cali_inc_data.head(10)

,incentive_count_residential,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,electricity_price_residential
108,75,21,9,0,38,0,10,14.72,16.99
109,75,21,9,0,38,0,10,14.72,16.99
110,75,21,9,0,38,0,10,14.72,16.99
111,75,21,9,0,38,0,10,14.72,16.99
112,75,21,9,0,38,0,10,14.72,16.99
113,75,21,9,0,38,0,10,14.72,16.99
114,75,21,9,0,38,0,10,14.72,16.99
115,75,21,9,0,38,0,10,14.72,16.99
116,75,21,9,0,38,0,10,14.72,16.99
117,75,21,9,0,38,0,10,14.72,16.99


In [628]:
# Looking into missing values of California demographic variables

cali_inc_data.isnull().sum()

incentive_count_residential      0
net_metering                     0
feedin_tariff                    0
cooperate_tax                    0
property_tax                     0
sales_tax                        0
rebate                           0
avg_electricity_retail_rate      0
electricity_price_residential    0
dtype: int64

    ** The incentive variables of the State of California does not have any missing values as shown above
    
    ** Likewise, the electricity retail rate variables of the State of California does not have any missing values 

In [10]:
# For convenince, the clean incentive data slice is labeled as clean

clean_cali_inc_data = cali_inc_data.copy()
clean_cali_inc_data

,incentive_count_residential,net_metering,feedin_tariff,cooperate_tax,property_tax,sales_tax,rebate,avg_electricity_retail_rate,electricity_price_residential
108,75,21,9,0,38,0,10,14.72,16.99
109,75,21,9,0,38,0,10,14.72,16.99
110,75,21,9,0,38,0,10,14.72,16.99
111,75,21,9,0,38,0,10,14.72,16.99
112,75,21,9,0,38,0,10,14.72,16.99
...,...,...,...,...,...,...,...,...,...
70868,75,21,9,0,38,0,10,14.72,16.99
70869,75,21,9,0,38,0,10,14.72,16.99
70870,75,21,9,0,38,0,10,14.72,16.99
70871,75,21,9,0,38,0,10,14.72,16.99


      ** At this stage, we have clean California incentive and electricity retail rate data
     

### 2. Statewide Demographic Parameters 

In [12]:
demographic_parameters = [
'county',
'average_household_income',
'gini_index',
'per_capita_income',                                                    
'population_density',                                                   
'education_less_than_high_school_rate',         
'education_high_school_graduate_rate',         
'education_college_rate',                       
'education_bachelor_rate',                      
'education_master_rate',                        
'education_professional_school_rate',           
'education_doctoral_rate',                      
'race_white_rate',                              
'race_black_africa_rate',                       
'race_indian_alaska_rate',                      
'race_asian_rate',                              
'race_islander_rate',                          
'race_other_rate',                              
'race_two_more_rate',
'poverty_family_below_poverty_level_rate',      
'heating_fuel_gas_rate',                        
'heating_fuel_electricity_rate',                
'heating_fuel_fuel_oil_kerosene_rate',          
'heating_fuel_coal_coke_rate',                  
'heating_fuel_solar_rate',                      
'heating_fuel_other_rate',                      
'heating_fuel_none_rate',                       
'household_count',                                
'average_household_size',                      
'housing_unit_count',                             
'housing_unit_occupied_count',                    
'housing_unit_median_value',                   
'housing_unit_median_gross_rent',
'age_18_24_rate',                               
'age_25_34_rate',                               
'age_more_than_85_rate',                        
'age_75_84_rate',                               
'age_35_44_rate',                               
'age_45_54_rate',                               
'age_65_74_rate',                               
'age_55_64_rate',                               
'age_10_14_rate',                               
'age_15_17_rate',                               
'age_5_9_rate',                                 
'household_type_family_rate',                   
'dropout_16_19_inschool_rate',                 
'occupation_construction_rate',                 
'occupation_public_rate',                       
'occupation_information_rate',                  
'occupation_finance_rate',                      
'occupation_education_rate',                    
'occupation_administrative_rate',               
'occupation_manufacturing_rate',                
'occupation_wholesale_rate',                    
'occupation_retail_rate',                       
'occupation_transportation_rate',               
'occupation_arts_rate',                        
'occupation_agriculture_rate',                  
'occupancy_vacant_rate',                        
'occupancy_owner_rate',                         
'mortgage_with_rate',                          
'transportation_home_rate',                     
'transportation_car_alone_rate',                
'transportation_walk_rate',                     
'transportation_carpool_rate',                  
'transportation_motorcycle_rate',               
'transportation_bicycle_rate',                  
'transportation_public_rate',                   
'travel_time_less_than_10_rate',               
'travel_time_10_19_rate',                       
'travel_time_20_29_rate',                       
'travel_time_30_39_rate',                       
'travel_time_40_59_rate',                       
'travel_time_60_89_rate',                       
'health_insurance_public_rate',                 
'health_insurance_none_rate',                                 
]

len(demographic_parameters)

76

In [13]:
# Slicing the demographic variables and reviewing the first ten elements

cali_demo_data = cali_data[demographic_parameters]
cali_demo_data.head(10)

,county,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,...,transportation_bicycle_rate,transportation_public_rate,travel_time_less_than_10_rate,travel_time_10_19_rate,travel_time_20_29_rate,travel_time_30_39_rate,travel_time_40_59_rate,travel_time_60_89_rate,health_insurance_public_rate,health_insurance_none_rate
108,Los Angeles County,88640.331010,0.4576,19648.0,11309.820,0.047858,0.225010,0.407565,0.146662,0.130838,...,0.021792,0.059322,0.164557,0.281100,0.181580,0.153208,0.165430,0.026626,0.186557,0.047268
109,Los Angeles County,99115.358593,0.4084,29555.0,8865.547,0.102294,0.203658,0.389957,0.162430,0.114693,...,0.027196,0.013375,0.096819,0.184752,0.200655,0.199252,0.230589,0.065949,0.180734,0.102952
110,Los Angeles County,414706.238185,0.5052,124712.0,2624.595,0.064271,0.149701,0.118164,0.289820,0.114970,...,0.000000,0.000610,0.212648,0.248221,0.132806,0.130435,0.109881,0.109091,0.209055,0.066238
111,Los Angeles County,107896.201580,0.5405,47022.0,11027.470,0.066475,0.269809,0.218769,0.251698,0.078411,...,0.000000,0.016873,0.128903,0.300641,0.234988,0.193755,0.121297,0.020416,0.309609,0.062601
112,Los Angeles County,123826.462548,0.5640,67095.0,12906.290,0.026752,0.119819,0.270535,0.340995,0.122080,...,0.024636,0.022956,0.143827,0.230864,0.238272,0.162346,0.200617,0.009877,0.275600,0.091574
113,Los Angeles County,50569.517885,0.4330,16951.0,12753.500,0.255090,0.337725,0.282236,0.063074,0.041118,...,0.000000,0.091563,0.046680,0.295858,0.173570,0.224195,0.101907,0.145957,0.372544,0.215128
114,Los Angeles County,130238.329911,0.5376,51988.0,9255.900,0.036670,0.097910,0.263293,0.320865,0.161349,...,0.000000,0.017277,0.141155,0.358547,0.150685,0.091721,0.159023,0.095295,0.207902,0.138690
115,Los Angeles County,102484.424449,0.4666,46837.0,19410.790,0.042698,0.113861,0.214109,0.340140,0.144802,...,0.015290,0.043455,0.134703,0.330414,0.168529,0.183026,0.175778,0.007551,0.150986,0.119670
116,Los Angeles County,62724.734694,0.4400,18892.0,8364.018,0.244304,0.290823,0.230380,0.179430,0.054114,...,0.004722,0.025184,0.018707,0.282202,0.140032,0.263495,0.176376,0.102619,0.398520,0.134559
117,Los Angeles County,198889.278901,0.5685,95105.0,10101.790,0.018913,0.043499,0.164775,0.348936,0.196217,...,0.000000,0.013936,0.165555,0.224009,0.149863,0.251079,0.130247,0.057670,0.244460,0.044316


In [14]:
# Looking into missing values of California demographic variables

cali_demo_data.isnull().sum()

county                           0
average_household_income        84
gini_index                      81
per_capita_income               50
population_density              22
                                ..
travel_time_30_39_rate          66
travel_time_40_59_rate          66
travel_time_60_89_rate          66
health_insurance_public_rate    65
health_insurance_none_rate      65
Length: 76, dtype: int64

    ** All the missing values of demographic variables in the State of California are listed above
    
    ** Considering the number of samples we have, over 8,000 samples, the number of missing values are manageable
    
    ** Hence these missing values will be filled with some values of the corresponding variable. 
    
    ** The values will be filled with the mean values of corresponding county. 
  

In [15]:
# Extracting the demographic variables with missing values

demo_var = cali_demo_data.isnull().sum() > 0
mis_val_var = demo_var[demo_var == True].keys()
mis_val_var

Index(['average_household_income', 'gini_index', 'per_capita_income',
       'population_density', 'education_less_than_high_school_rate',
       'education_high_school_graduate_rate', 'education_college_rate',
       'education_bachelor_rate', 'education_master_rate',
       'education_professional_school_rate', 'education_doctoral_rate',
       'race_white_rate', 'race_black_africa_rate', 'race_indian_alaska_rate',
       'race_asian_rate', 'race_islander_rate', 'race_other_rate',
       'race_two_more_rate', 'poverty_family_below_poverty_level_rate',
       'heating_fuel_gas_rate', 'heating_fuel_electricity_rate',
       'heating_fuel_fuel_oil_kerosene_rate', 'heating_fuel_coal_coke_rate',
       'heating_fuel_solar_rate', 'heating_fuel_other_rate',
       'heating_fuel_none_rate', 'average_household_size',
       'housing_unit_median_value', 'housing_unit_median_gross_rent',
       'age_18_24_rate', 'age_25_34_rate', 'age_more_than_85_rate',
       'age_75_84_rate', 'age_35_44_rate

In [16]:
#  Extracting State of California counties and checking them with missing demographic variable values

cali_county = cali_demo_data['county'].unique().tolist()
cali_county

['Los Angeles County',
 'San Mateo County',
 'San Joaquin County',
 'San Diego County',
 'Tuolumne County',
 'Marin County',
 'Riverside County',
 'San Francisco County',
 'Contra Costa County',
 'Butte County',
 'Orange County',
 'San Luis Obispo County',
 'San Benito County',
 'Alameda County',
 'Kern County',
 'Napa County',
 'Santa Clara County',
 'Tulare County',
 'Humboldt County',
 'Placer County',
 'Sonoma County',
 'Fresno County',
 'Monterey County',
 'Plumas County',
 'Merced County',
 'Stanislaus County',
 'Calaveras County',
 'Santa Cruz County',
 'Yuba County',
 'San Bernardino County',
 'Santa Barbara County',
 'Solano County',
 'Sacramento County',
 'Madera County',
 'Tehama County',
 'Lake County',
 'Imperial County',
 'Del Norte County',
 'Kings County',
 'Inyo County',
 'Lassen County',
 'Ventura County',
 'Nevada County',
 'Mendocino County',
 'Modoc County',
 'Colusa County',
 'El Dorado County',
 'Sutter County',
 'Alpine County',
 'Yolo County',
 'Siskiyou County

    ** In the above sections, all the demographic variables with missing values are listed,
    
    ** Moreover, all the counties with missing demographic values are listed above,
    
    ** As shown below, a function is used to fill the values with the mean values of corresponding county, 

In [18]:
# Function filling missing values of California environmental variables based on the mean of respective county

def fill_miss_data(dataset, mis_val_var, mis_val_county):
    
    # Iterate over the list of counties with missing values of environmental variables
    for i in range(len(mis_val_county)):
        
        # Slicing the dataset to a county level
        data = dataset[dataset['county'] == mis_val_county[i]]
        
        #Iterating over the missing variables with in the sliced county
        for j in range(len(mis_val_var)):
            
            var = mis_val_var[j]
            mean = data[var].mean()              # Computation of the mean value of an demographic variable
            dataset[var].fillna(mean, inplace=True) # Filling missing values with mean value of respective county
            
    return dataset

In [20]:
# Appyiing the function to fill the values and checking for missing values again

cali_demo_data = fill_miss_data(cali_demo_data, mis_val_var, cali_county)
cali_demo_data.isnull().any().any()

False

In [21]:
# Since the demographic variables do not have missing values anymore, it is labeled as clean for convenience

clean_cali_demo_data = cali_demo_data.copy()
clean_cali_demo_data.head()

,county,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,...,transportation_bicycle_rate,transportation_public_rate,travel_time_less_than_10_rate,travel_time_10_19_rate,travel_time_20_29_rate,travel_time_30_39_rate,travel_time_40_59_rate,travel_time_60_89_rate,health_insurance_public_rate,health_insurance_none_rate
108,Los Angeles County,88640.331010,0.4576,19648.0,11309.820,0.047858,0.225010,0.407565,0.146662,0.130838,...,0.021792,0.059322,0.164557,0.281100,0.181580,0.153208,0.165430,0.026626,0.186557,0.047268
109,Los Angeles County,99115.358593,0.4084,29555.0,8865.547,0.102294,0.203658,0.389957,0.162430,0.114693,...,0.027196,0.013375,0.096819,0.184752,0.200655,0.199252,0.230589,0.065949,0.180734,0.102952
110,Los Angeles County,414706.238185,0.5052,124712.0,2624.595,0.064271,0.149701,0.118164,0.289820,0.114970,...,0.000000,0.000610,0.212648,0.248221,0.132806,0.130435,0.109881,0.109091,0.209055,0.066238
111,Los Angeles County,107896.201580,0.5405,47022.0,11027.470,0.066475,0.269809,0.218769,0.251698,0.078411,...,0.000000,0.016873,0.128903,0.300641,0.234988,0.193755,0.121297,0.020416,0.309609,0.062601
112,Los Angeles County,123826.462548,0.5640,67095.0,12906.290,0.026752,0.119819,0.270535,0.340995,0.122080,...,0.024636,0.022956,0.143827,0.230864,0.238272,0.162346,0.200617,0.009877,0.275600,0.091574


     ** At this part of the project, the State of California demographical variables do not have missing values
     
     ** Clean California demographical variables

### 3. Statewide Environmental Condition Parameters

In [23]:
envt_parameters = [
    'county',
'heating_design_temperature',                  
'cooling_design_temperature',                  
'earth_temperature_amplitude',             
'frost_days',                                 
'air_temperature',                             
'relative_humidity',                          
'daily_solar_radiation',                       
'atmospheric_pressure',                        
'wind_speed',                                 
'earth_temperature',                           
'heating_degree_days',                         
'cooling_degree_days',
]
len(envt_parameters)

13

In [24]:
# Slicing the dataset to reflect the envornmental variables/parameters

cali_envt_data = cali_data[envt_parameters]
cali_envt_data.head(10)

,county,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
108,Los Angeles County,7.39,28.26,13.18,0.0,17.6,0.515,5.20,98.6,4.6,19.5,767.0,2841.0
109,Los Angeles County,7.39,28.26,13.18,0.0,17.6,0.515,5.20,98.6,4.6,19.5,767.0,2841.0
110,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
111,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
112,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
113,Los Angeles County,7.39,28.26,13.18,0.0,17.6,0.515,5.20,98.6,4.6,19.5,767.0,2841.0
114,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
115,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
116,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
117,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0


In [25]:
# Checking missing values of California environmental variables

cali_envt_data.isnull().sum() 

county                          0
heating_design_temperature     16
cooling_design_temperature     16
earth_temperature_amplitude    16
frost_days                     16
air_temperature                16
relative_humidity              16
daily_solar_radiation          16
atmospheric_pressure           16
wind_speed                     16
earth_temperature              16
heating_degree_days            16
cooling_degree_days            16
dtype: int64

In [26]:
# The fact they all have the same number of missing values implies they might be missing from the same places

# USING ONE OF THE VARIABLES  to check if the missing values are from the same places

# It is here shown that all the missing values are from the same places of different counties

cali_envt_data[cali_envt_data['wind_speed'].isnull() == True]

,county,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
633,Los Angeles County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,Los Angeles County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4539,San Mateo County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8229,San Diego County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14157,Marin County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17791,San Francisco County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17979,Contra Costa County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19220,Orange County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19966,San Luis Obispo County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23017,Alameda County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Listing the counties in the State of California with missing environmental variable values

mis_val_county = cali_envt_data[cali_envt_data['wind_speed'].isnull() == True]['county'].unique().tolist()
mis_val_county

['Los Angeles County',
 'San Mateo County',
 'San Diego County',
 'Marin County',
 'San Francisco County',
 'Contra Costa County',
 'Orange County',
 'San Luis Obispo County',
 'Alameda County',
 'Humboldt County',
 'Sonoma County',
 'Monterey County',
 'Santa Cruz County',
 'Del Norte County',
 'Mendocino County']

In [28]:
# These are the environmental variables with missing values in the State of California

mis_val_var = envt_parameters[1:]
mis_val_var

['heating_design_temperature',
 'cooling_design_temperature',
 'earth_temperature_amplitude',
 'frost_days',
 'air_temperature',
 'relative_humidity',
 'daily_solar_radiation',
 'atmospheric_pressure',
 'wind_speed',
 'earth_temperature',
 'heating_degree_days',
 'cooling_degree_days']

    ** As shown in the above , we have 16 missing values from each of those environmental variables listed above,
    
    ** Those missing values belong to different counties,
    
    ** Hence, the mean values of corresponding county are used to fill the missing values.

In [29]:
# Running the previous function to fill the missing values and checking for missing values again

cali_envt_data = fill_miss_data(cali_envt_data, mis_val_var, mis_val_county )
cali_envt_data.isnull().sum()

county                         0
heating_design_temperature     0
cooling_design_temperature     0
earth_temperature_amplitude    0
frost_days                     0
air_temperature                0
relative_humidity              0
daily_solar_radiation          0
atmospheric_pressure           0
wind_speed                     0
earth_temperature              0
heating_degree_days            0
cooling_degree_days            0
dtype: int64

In [30]:
# Finalize and copy the clean environmental variables and labeling it as clean

clean_cali_envt_data = cali_envt_data.copy()
clean_cali_envt_data.head()

,county,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days
108,Los Angeles County,7.39,28.26,13.18,0.0,17.6,0.515,5.20,98.6,4.6,19.5,767.0,2841.0
109,Los Angeles County,7.39,28.26,13.18,0.0,17.6,0.515,5.20,98.6,4.6,19.5,767.0,2841.0
110,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
111,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0
112,Los Angeles County,2.35,33.22,22.20,3.0,17.2,0.401,5.41,94.7,4.4,19.8,1208.0,2815.0


     ** At this stage of the implementation, the California environemental variables do not have missing values
     ** Clean California environmental data

### 4. Statewide Political Views Parameters


In [39]:
pol_parameters = [
    
'voting_2016_dem_percentage',                     
'voting_2016_gop_percentage',                     
'voting_2016_dem_win',                            
'voting_2012_dem_percentage',                 
'voting_2012_gop_percentage',                
'voting_2012_dem_win'                           
]

len(pol_parameters)

6

In [35]:
# Looking into the missing values of Poltical view data values in the State of California

# As shown here below, this part of the dataset fortunately does not have missing values

cali_pol_data = cali_data[pol_parameters]
cali_pol_data.isnull().sum()

voting_2016_dem_percentage    0
voting_2016_gop_percentage    0
voting_2016_dem_win           0
voting_2012_dem_percentage    0
voting_2012_gop_percentage    0
voting_2012_dem_win           0
dtype: int64

In [37]:
# However, mapping the True and False value to a corresponding binary variables enhances applying computations

# Mapping True and False values of our attributes into 1 and 0 respectively

vot_map = {True: 1, False: 0}

cali_pol_data['voting_2016_dem_win'] = cali_pol_data['voting_2016_dem_win'].map(vot_map)
cali_pol_data['voting_2012_dem_win'] = cali_pol_data['voting_2016_dem_win'].map(vot_map)
cali_pol_data.head()

,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
108,0.714072,0.233887,1,0.689,0.288,1
109,0.714072,0.233887,1,0.689,0.288,1
110,0.714072,0.233887,1,0.689,0.288,1
111,0.714072,0.233887,1,0.689,0.288,1
112,0.714072,0.233887,1,0.689,0.288,1


In [40]:
# Finalize and copy the clean political variables and labeling it as clean

clean_cali_pol_data = cali_pol_data.copy()
clean_cali_pol_data

,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
108,0.714072,0.233887,1,0.689,0.288,1
109,0.714072,0.233887,1,0.689,0.288,1
110,0.714072,0.233887,1,0.689,0.288,1
111,0.714072,0.233887,1,0.689,0.288,1
112,0.714072,0.233887,1,0.689,0.288,1
...,...,...,...,...,...,...
70868,0.326794,0.617123,0,0.358,0.613,0
70869,0.326794,0.617123,0,0.358,0.613,0
70870,0.326794,0.617123,0,0.358,0.613,0
70871,0.326794,0.617123,0,0.358,0.613,0


      * The political views variables from the State of California is clean and ready to be processed at this stage

## Merging Clean Environmental and Socio-economic Dataset Slices

In [74]:
# To perform computations on all the slices combined, all the slices are merged at this section

# Moreover, the dimensions of the dataset are also checked for consistency

slices = [clean_cali_inc_data, clean_cali_demo_data, clean_cali_envt_data, clean_cali_pol_data]
dataset = pd.concat(slices, axis=1, sort=False)
dataset.drop('county', axis=1, inplace=True)
dataset.shape

(8056, 102)

In [75]:
# Further checking for missing values and consistency checking

dataset.isnull().sum()

incentive_count_residential    0
net_metering                   0
feedin_tariff                  0
cooperate_tax                  0
property_tax                   0
                              ..
voting_2016_gop_percentage     0
voting_2016_dem_win            0
voting_2012_dem_percentage     0
voting_2012_gop_percentage     0
voting_2012_dem_win            0
Length: 102, dtype: int64

     ** At this stage, the environmental and socio-economic variables reflecting the State of California are clean and ready for further regression or classification processing

  --------------------------------------------------------------------------------------------------------------
    
    ** In the following section, a classification algorithm is applied to the clean dataset
    
    ** This algorithm will classify area with/without solar panels installed

# Classification

###     A.   Feature Selection 

      ** Upto this stage, we have been cleaning socio-economic dataset for missing values in State of California 
      
      ** This clean State of California's socio-economic dataset contains 8056 data records with 102 dimensions
      
      ** Feature selection is applied at this stage to drop features with little contribution for classification 

In [76]:
# Removing features with zero standard deviation - having values that does not change 

print("Original dataset shape - ", dataset.shape)

# Remove Constant Features
constant_features = [feat for feat in dataset.columns if (dataset[feat].std() == 0 or dataset[feat].min() == dataset[feat].max())]

print('\nThe following features have constant with little contribution for classification, Hence they are dropped \n\n', constant_features)
print('\n')

# Dropping all the data with constant values 

dataset.drop(labels=constant_features, axis=1, inplace=True)
print("Reduced feature California socio economic dataset shape -",dataset.shape)

Original dataset shape -  (8056, 102)

The following features have constant with little contribution for classification, Hence they are dropped 

 ['incentive_count_residential', 'net_metering', 'feedin_tariff', 'cooperate_tax', 'property_tax', 'sales_tax', 'rebate', 'avg_electricity_retail_rate', 'electricity_price_residential']


Reduced feature California socio economic dataset shape - (8056, 93)


In [77]:
# Looking into the distribution of the data

dataset.describe()

,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,...,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
count,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,...,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000
mean,85433.476192,0.420459,31141.920827,8600.291584,0.190524,0.208683,0.294265,0.193033,0.075635,0.022714,...,4.084094,18.539812,1226.900033,2566.922326,0.620551,0.323369,0.918942,0.598046,0.378421,0.918942
std,43743.271886,0.059864,18650.855284,9651.212644,0.159264,0.080379,0.090287,0.113595,0.066832,0.027514,...,0.392103,2.358721,403.933648,474.024755,0.123119,0.121425,0.272940,0.119524,0.121100,0.272940
min,9040.000000,0.012800,470.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.400000,10.000000,632.000000,1051.000000,0.213395,0.094425,0.000000,0.279000,0.133000,0.000000
25%,55694.517065,0.380400,17799.000000,2669.763750,0.062448,0.152630,0.231739,0.096597,0.023827,0.004127,...,3.700000,16.600000,889.000000,2248.000000,0.509602,0.233887,1.000000,0.492000,0.288000,1.000000
50%,75782.095234,0.416500,26924.000000,6315.999500,0.140160,0.213870,0.295907,0.180932,0.056847,0.013443,...,4.000000,19.500000,1208.000000,2815.000000,0.647662,0.295744,1.000000,0.636000,0.338000,1.000000
75%,103458.331336,0.454100,39263.500000,11045.155000,0.283611,0.266950,0.357706,0.280578,0.111044,0.030362,...,4.400000,19.800000,1370.000000,2853.000000,0.714072,0.432801,1.000000,0.689000,0.480000,1.000000
max,537130.215827,0.731700,181215.000000,173333.700000,1.000000,0.522634,1.000000,1.000000,1.000000,0.235900,...,5.600000,25.200000,3657.000000,4089.000000,0.855317,0.726732,1.000000,0.834000,0.697000,1.000000


     ** It is shown in the above data distribution that the difference between the feature values has high proportional difference
     
     ** For example, average_household_income max value is 537130.22, however gini_index has max value of 0.73. 
     
     ** Since proportion difference leads our classifier for possible bias, normalization is crucial for the data

### B. Dataset Normalization

In [87]:
## Applying min max normalization to the dataset

# This normalization technique will normalize the values and retain our dataset distribution 

data_norm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
data_norm.head()

,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,...,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
108,0.150732,0.618723,0.106105,0.065249,0.047858,0.430530,0.407565,0.146662,0.130838,0.086712,...,0.545455,0.625000,0.044628,0.589203,0.779965,0.22056,1.0,0.738739,0.274823,1.0
109,0.170568,0.550285,0.160917,0.051147,0.102294,0.389676,0.389957,0.162430,0.114693,0.057818,...,0.545455,0.625000,0.044628,0.589203,0.779965,0.22056,1.0,0.738739,0.274823,1.0
110,0.768176,0.684935,0.687388,0.015142,0.064271,0.286435,0.118164,0.289820,0.114970,0.895199,...,0.454545,0.644737,0.190413,0.580645,0.779965,0.22056,1.0,0.738739,0.274823,1.0
111,0.187196,0.734038,0.257556,0.063620,0.066475,0.516248,0.218769,0.251698,0.078411,0.247767,...,0.454545,0.644737,0.190413,0.580645,0.779965,0.22056,1.0,0.738739,0.274823,1.0
112,0.217361,0.766727,0.368613,0.074459,0.026752,0.229260,0.270535,0.340995,0.122080,0.372157,...,0.454545,0.644737,0.190413,0.580645,0.779965,0.22056,1.0,0.738739,0.274823,1.0


In [88]:
# Looking into distribution of data in all variables

data_norm.describe()

,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,...,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
count,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,...,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000,8056.000000
mean,0.144660,0.567060,0.169697,0.049617,0.190524,0.399291,0.294265,0.193033,0.075635,0.096285,...,0.310952,0.561830,0.196661,0.498987,0.634277,0.362078,0.918942,0.574858,0.435143,0.918942
std,0.082833,0.083271,0.103189,0.055680,0.159264,0.153796,0.090287,0.113595,0.066832,0.116636,...,0.178228,0.155179,0.133532,0.156032,0.191797,0.192035,0.272940,0.215359,0.214716,0.272940
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.088346,0.511337,0.095875,0.015402,0.062448,0.292040,0.231739,0.096597,0.023827,0.017493,...,0.136364,0.434211,0.084959,0.394009,0.461437,0.220560,1.000000,0.383784,0.274823,1.000000
50%,0.126384,0.561552,0.146361,0.036438,0.140160,0.409216,0.295907,0.180932,0.056847,0.056986,...,0.272727,0.625000,0.190413,0.580645,0.676510,0.318389,1.000000,0.643243,0.363475,1.000000
75%,0.178792,0.613854,0.214631,0.063722,0.283611,0.510778,0.357706,0.280578,0.111044,0.128706,...,0.454545,0.644737,0.243967,0.593153,0.779965,0.535145,1.000000,0.738739,0.615248,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


    ** The dataset is normalized using min-max normalization as shown and retains previous distribution 
    
    ** Furthermore, min-max retains information embedded in the dataset

In [93]:
# Extracting the indices of each data record in the normalized data

# These indices will help in assigning labels to each data record

ind = data_norm.index

In [96]:
# The Solar_System_Count is used as a label for our dataset 

      #.  solar_system_count > 0,  label = 0
      #.  solar_system_count = 0,  label = 0

labels = data['solar_system_count'][ind].values
labels = np.where(labels > 0, 1, 0)
labels

array([1, 1, 1, ..., 0, 1, 0])

In [102]:
# Checking if the number of labels is equal to the number of records

# As shown below, the number of labels match the number of data records

print("The dataset has", data_norm.shape[0], "data records.")
print("The labels array has", len(labels), "labels.")

The dataset has 8056 data records.
The labels array has 8056 labels.


### C. Splitting the dataset into Train, Validation and Test Subsets

     ** In this section, the dataset is split into train, validation and test subsets
     
                 ^^. 60% training subset
                 ^^. 20% validation subset
                 ^^. 20% test subset

In [103]:
# split into training and validations subsets

# First splitting the dataset into 60% train and 40% validation

from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(data_norm, labels, test_size = 0.4, random_state = 42, shuffle = True, stratify = labels)

In [104]:
# Split the validation data into 50% validation and 50% test subsets

X_val, X_test, y_val, y_test = train_test_split(X_validation, y_validation, test_size=0.5, random_state=42, shuffle=True ,stratify=y_validation )

In [105]:
# A function to print the accuracy results for every combination of parameters we will use for our classfier

def print_results(results):
    print("Best Params: {}\n".format(results.best_params_))   
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean,3), round(std*2, 3), params))

In [900]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [300, 350],
    'max_depth': [32,None],
    'max_features': [2, 3, 4],
    
    
}

cv = GridSearchCV(rf, parameters, cv=10, scoring='accuracy')
cv.fit(X_train, y_train)


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [901]:
print_results(cv)

Best Params: {'max_depth': None, 'max_features': 4, 'n_estimators': 300}

0.971 (+/-0.006) for {'max_depth': 32, 'max_features': 2, 'n_estimators': 250}
0.971 (+/-0.006) for {'max_depth': 32, 'max_features': 2, 'n_estimators': 300}
0.971 (+/-0.005) for {'max_depth': 32, 'max_features': 2, 'n_estimators': 350}
0.972 (+/-0.007) for {'max_depth': 32, 'max_features': 3, 'n_estimators': 250}
0.972 (+/-0.007) for {'max_depth': 32, 'max_features': 3, 'n_estimators': 300}
0.971 (+/-0.007) for {'max_depth': 32, 'max_features': 3, 'n_estimators': 350}
0.972 (+/-0.006) for {'max_depth': 32, 'max_features': 4, 'n_estimators': 250}
0.971 (+/-0.006) for {'max_depth': 32, 'max_features': 4, 'n_estimators': 300}
0.972 (+/-0.006) for {'max_depth': 32, 'max_features': 4, 'n_estimators': 350}
0.971 (+/-0.006) for {'max_depth': None, 'max_features': 2, 'n_estimators': 250}
0.971 (+/-0.005) for {'max_depth': None, 'max_features': 2, 'n_estimators': 300}
0.971 (+/-0.007) for {'max_depth': None, 'max_feature

In [902]:
# PREDICTION ON VALIDATION DATA

val_prediction = cv.predict(X_val)

In [903]:
# VALIDATION DATA ACCURACY

print('Accuracy score for Testing Dataset = ', accuracy_score(y_val, val_prediction))

Accuracy score for Testing Dataset =  0.9745499689633768


In [904]:
# PREDICTION ON TEST DATA

test_prediction = cv.predict(X_test)

In [905]:
# TEST DATA ACCURACY

print('Accuracy score for Testing Dataset = ', accuracy_score(y_test, test_prediction))

Accuracy score for Testing Dataset =  0.9745657568238213


In [916]:
# PREDICTION ON OVERALL DATASET

tot_predictions = cv.predict(data_norm)

In [917]:
# OVERALL DATA ACCURACY

print('Accuracy score for Testing Dataset = ', accuracy_score(labels, tot_predictions))

Accuracy score for Testing Dataset =  0.9895729890764647


In [937]:
# COPYING NORMALIZED DATA TO REFLECT CLASSIFIED DATA

class_data = data_norm.copy()
class_data.shape

(8056, 94)

In [938]:
# EXTRACTING DATA RECORDS WITH SOLAR PANELS ACCORDING TO OUR CLASSFIER RESULT

class_data['prediction'] = tot_predictions
class_data = class_data[class_data['prediction'] == 1]
class_data.drop('prediction', axis=1)
class_data.shape

,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,education_doctoral_rate,race_white_rate,race_black_africa_rate,race_indian_alaska_rate,race_asian_rate,race_islander_rate,race_other_rate,race_two_more_rate,poverty_family_below_poverty_level_rate,heating_fuel_gas_rate,heating_fuel_electricity_rate,heating_fuel_fuel_oil_kerosene_rate,heating_fuel_coal_coke_rate,heating_fuel_solar_rate,heating_fuel_other_rate,heating_fuel_none_rate,household_count,average_household_size,housing_unit_count,housing_unit_occupied_count,housing_unit_median_value,housing_unit_median_gross_rent,age_18_24_rate,age_25_34_rate,age_more_than_85_rate,age_75_84_rate,age_35_44_rate,age_45_54_rate,age_65_74_rate,age_55_64_rate,age_10_14_rate,age_15_17_rate,age_5_9_rate,household_type_family_rate,dropout_16_19_inschool_rate,occupation_construction_rate,occupation_public_rate,occupation_information_rate,occupation_finance_rate,occupation_education_rate,occupation_administrative_rate,occupation_manufacturing_rate,occupation_wholesale_rate,occupation_retail_rate,occupation_transportation_rate,occupation_arts_rate,occupation_agriculture_rate,occupancy_vacant_rate,occupancy_owner_rate,mortgage_with_rate,transportation_home_rate,transportation_car_alone_rate,transportation_walk_rate,transportation_carpool_rate,transportation_motorcycle_rate,transportation_bicycle_rate,transportation_public_rate,travel_time_less_than_10_rate,travel_time_10_19_rate,travel_time_20_29_rate,travel_time_30_39_rate,travel_time_40_59_rate,travel_time_60_89_rate,health_insurance_public_rate,health_insurance_none_rate,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win
108,0.150732,0.618723,0.106105,0.065249,0.047858,0.430530,0.407565,0.146662,0.130838,0.086712,0.078799,0.658069,0.015244,0.004775,0.180700,0.015370,0.102679,0.053075,0.030888,0.788328,0.179443,0.000000,0.000000,0.150193,0.013862,0.034193,0.133893,0.255441,0.119329,0.133893,0.201709,0.371914,0.445106,0.134643,0.118340,0.060354,0.157821,0.150388,0.052083,0.181103,0.162808,0.034557,0.198883,0.676829,1.000000,0.029083,0.059445,0.032098,0.156575,0.370179,0.115949,0.198310,0.053394,0.256920,0.038891,0.152286,0.009207,0.021313,0.663257,0.677378,0.092234,0.616582,0.186037,0.092327,0.032486,0.037500,0.059322,0.164557,0.281100,0.293322,0.298943,0.165430,0.026626,0.186557,0.067549,0.857143,0.530498,0.450137,0.000000,0.720,0.434884,0.805556,0.863388,0.545455,0.625000,0.044628,0.589203,0.779965,0.220560,1.0,0.738739,0.274823,1.0
109,0.170568,0.550285,0.160917,0.051147,0.102294,0.389676,0.389957,0.162430,0.114693,0.057818,0.048597,0.652914,0.034470,0.000000,0.136572,0.000000,0.143073,0.048637,0.060877,0.891746,0.104195,0.000000,0.000000,0.000000,0.000000,0.008387,0.172382,0.270332,0.159003,0.172382,0.213015,0.394444,0.152347,0.188062,0.086257,0.079422,0.230334,0.286491,0.079485,0.224220,0.342386,0.042230,0.347449,0.833559,1.000000,0.053525,0.118860,0.105022,0.120219,0.282307,0.090826,0.335291,0.069384,0.143439,0.354596,0.084562,0.000000,0.054383,0.738324,0.730503,0.057215,0.838094,0.054391,0.085396,0.000000,0.046800,0.013375,0.096819,0.184752,0.324135,0.388784,0.230589,0.065949,0.180734,0.147126,0.857143,0.530498,0.450137,0.000000,0.720,0.434884,0.805556,0.863388,0.545455,0.625000,0.044628,0.589203,0.779965,0.220560,1.0,0.738739,0.274823,1.0
110,0.768176,0.684935,0.687388,0.015142,0.064271,0.286435,0.118164,0.289820,0.114970,0.895199,0.189207,0.853829,0.011941,0.000000,0.062435,0.000000,0.022079,0.056456,0.018315,0.

(7879, 94)

    ** Random Forest Classifier has classified our data records into areas with solar system and areas without
    
    ** In the following section, A REGRESSION ALGORITHM WILL BE APPLIED TO DATA RECORDS WITH SOLAR SYSTEMS ONLY 

## REGRESSION

In [932]:
reg_labels = data['solar_system_count'][class_data.index]
reg_labels

108       69.0
109      100.0
110       72.0
111       44.0
112       21.0
         ...  
69082     12.0
70866    346.0
70867      8.0
70868     53.0
70871    219.0
Name: solar_system_count, Length: 7879, dtype: float64

In [ ]:
class

In [921]:
data_norm.head()

,average_household_income,gini_index,per_capita_income,population_density,education_less_than_high_school_rate,education_high_school_graduate_rate,education_college_rate,education_bachelor_rate,education_master_rate,education_professional_school_rate,education_doctoral_rate,race_white_rate,race_black_africa_rate,race_indian_alaska_rate,race_asian_rate,race_islander_rate,race_other_rate,race_two_more_rate,poverty_family_below_poverty_level_rate,heating_fuel_gas_rate,heating_fuel_electricity_rate,heating_fuel_fuel_oil_kerosene_rate,heating_fuel_coal_coke_rate,heating_fuel_solar_rate,heating_fuel_other_rate,heating_fuel_none_rate,household_count,average_household_size,housing_unit_count,housing_unit_occupied_count,housing_unit_median_value,housing_unit_median_gross_rent,age_18_24_rate,age_25_34_rate,age_more_than_85_rate,age_75_84_rate,age_35_44_rate,age_45_54_rate,age_65_74_rate,age_55_64_rate,age_10_14_rate,age_15_17_rate,age_5_9_rate,household_type_family_rate,dropout_16_19_inschool_rate,occupation_construction_rate,occupation_public_rate,occupation_information_rate,occupation_finance_rate,occupation_education_rate,occupation_administrative_rate,occupation_manufacturing_rate,occupation_wholesale_rate,occupation_retail_rate,occupation_transportation_rate,occupation_arts_rate,occupation_agriculture_rate,occupancy_vacant_rate,occupancy_owner_rate,mortgage_with_rate,transportation_home_rate,transportation_car_alone_rate,transportation_walk_rate,transportation_carpool_rate,transportation_motorcycle_rate,transportation_bicycle_rate,transportation_public_rate,travel_time_less_than_10_rate,travel_time_10_19_rate,travel_time_20_29_rate,travel_time_30_39_rate,travel_time_40_59_rate,travel_time_60_89_rate,health_insurance_public_rate,health_insurance_none_rate,heating_design_temperature,cooling_design_temperature,earth_temperature_amplitude,frost_days,air_temperature,relative_humidity,daily_solar_radiation,atmospheric_pressure,wind_speed,earth_temperature,heating_degree_days,cooling_degree_days,voting_2016_dem_percentage,voting_2016_gop_percentage,voting_2016_dem_win,voting_2012_dem_percentage,voting_2012_gop_percentage,voting_2012_dem_win,prediction
108,0.150732,0.618723,0.106105,0.065249,0.047858,0.430530,0.407565,0.146662,0.130838,0.086712,0.078799,0.658069,0.015244,0.004775,0.180700,0.01537,0.102679,0.053075,0.030888,0.788328,0.179443,0.000000,0.0,0.150193,0.013862,0.034193,0.133893,0.255441,0.119329,0.133893,0.201709,0.371914,0.445106,0.134643,0.118340,0.060354,0.157821,0.150388,0.052083,0.181103,0.162808,0.034557,0.198883,0.676829,1.0,0.029083,0.059445,0.032098,0.156575,0.370179,0.115949,0.198310,0.053394,0.256920,0.038891,0.152286,0.009207,0.021313,0.663257,0.677378,0.092234,0.616582,0.186037,0.092327,0.032486,0.037500,0.059322,0.164557,0.281100,0.293322,0.298943,0.165430,0.026626,0.186557,0.067549,0.857143,0.530498,0.450137,0.000000,0.720,0.434884,0.805556,0.863388,0.545455,0.625000,0.044628,0.589203,0.779965,0.22056,1.0,0.738739,0.274823,1.0,1
109,0.170568,0.550285,0.160917,0.051147,0.102294,0.389676,0.389957,0.162430,0.114693,0.057818,0.048597,0.652914,0.034470,0.000000,0.136572,0.00000,0.143073,0.048637,0.060877,0.891746,0.104195,0.000000,0.0,0.000000,0.000000,0.008387,0.172382,0.270332,0.159003,0.172382,0.213015,0.394444,0.152347,0.188062,0.086257,0.079422,0.230334,0.286491,0.079485,0.224220,0.342386,0.042230,0.347449,0.833559,1.0,0.053525,0.118860,0.105022,0.120219,0.282307,0.090826,0.335291,0.069384,0.143439,0.354596,0.084562,0.000000,0.054383,0.738324,0.730503,0.057215,0.838094,0.054391,0.085396,0.000000,0.046800,0.013375,0.096819,0.184752,0.324135,0.388784,0.230589,0.065949,0.180734,0.147126,0.857143,0.530498,0.450137,0.000000,0.720,0.434884,0.805556,0.863388,0.545455,0.625000,0.044628,0.589203,0.779965,0.22056,1.0,0.738739,0.274823,1.0,1
110,0.768176,0.684935,0.687388,0.015142,0.064271,0.286435,0.118164,0.289820,0.114970,0.895199,0.189207,0.853829,0.011941,0.000000,0.062435,0.00000,0.022079,0.056456,0.018315,0.801512,0.1

In [866]:
def evaluate(classifier, training_instances, training_labels):
    metrics = cross_validate(classifier, training_instances, training_labels, cv=10, 
                             n_jobs=-1, scoring=['accuracy'])
    print("Accuracy: %0.4f (+/- %0.4f)" % (metrics['test_accuracy'].mean(), metrics['test_accuracy'].std() * 2))
    print("Mean fit time: %0.4f ms" % (metrics['fit_time'].mean()))

In [867]:
voting = VotingClassifier([ 
    ('knn', OneVsRestClassifier(KNeighborsClassifier(n_neighbors = 4))),
    ('et', OneVsRestClassifier(ExtraTreesClassifier(n_estimators = 1000))),
    ('xgboost', XGBClassifier(learning_rate =0.1, n_estimators=1000, num_class = 12,
                             min_child_weight=1, gamma=0,subsample=0.8,colsample_bytree=0.8,
                             objective= 'multi:softmax', nthread=4,scale_pos_weight=1,
                             seed=27, early_stopping_rounds=70, verbose=False)),
    ('randomforest', RandomForestClassifier(n_estimators=1000)),
    ('ada', AdaBoostClassifier(n_estimators=1000))], voting='soft', weights =[1,2,1,2,1])

In [868]:
voting.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              OneVsRestClassifier(estimator=KNeighborsClassifier(algorithm='auto',
                                                                                 leaf_size=30,
                                                                                 metric='minkowski',
                                                                                 metric_params=None,
                                                                                 n_jobs=None,
                                                                                 n_neighbors=4,
                                                                                 p=2,
                                                                                 weights='uniform'),
                                                  n_jobs=None)),
                             ('et',
                              OneVsRestClassifier(estimator=ExtraTreesClassifier(bootstra

In [891]:
val_prediction = voting.predict(X_val)

In [892]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_val, val_prediction))

Accuracy score for Testing Dataset =  0.9770328988206083


In [893]:
test_prediction = voting.predict(X_test)

In [894]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_test, test_prediction))

Accuracy score for Testing Dataset =  0.9727047146401985


In [880]:
svc = SVC()
parameters = { 'kernel': ['linear','rbf'], 'C': [0.1, 1, 10]}

cv = GridSearchCV(svc, parameters, cv=10)
cv.fit(X_train, y_train)

print_results(cv)

GridSearchCV(cv=10, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

Best Params: {'C': 10, 'kernel': 'rbf'}

0.969 (+/-0.002) for {'C': 0.1, 'kernel': 'linear'}
0.969 (+/-0.002) for {'C': 0.1, 'kernel': 'rbf'}
0.969 (+/-0.002) for {'C': 1, 'kernel': 'linear'}
0.969 (+/-0.002) for {'C': 1, 'kernel': 'rbf'}
0.969 (+/-0.005) for {'C': 10, 'kernel': 'linear'}
0.971 (+/-0.005) for {'C': 10, 'kernel': 'rbf'}


In [885]:
val_prediction = cv.predict(X_val)

In [886]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_val, val_prediction))

Accuracy score for Testing Dataset =  0.9770328988206083


In [887]:
test_prediction = cv.predict(X_test)

In [888]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_test, test_prediction))

Accuracy score for Testing Dataset =  0.9727047146401985


In [895]:
mlp = MLPClassifier()
parameters = { 'hidden_layer_sizes': [(10,), (50,), (100,)],
                'activation': ['tanh', 'relu', 'linear'],
                 'learning_rate': ['constant', 'adaptive', 'invscaling']}

cv = GridSearchCV(mlp, parameters, cv=5)
cv.fit(X_train, y_train,)

print_results(cv)

GridSearchCV(cv=5, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state...uffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'acti

Best Params: {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}

0.97 (+/-0.006) for {'activation': 'tanh', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.971 (+/-0.005) for {'activation': 'tanh', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.97 (+/-0.006) for {'activation': 'tanh', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.97 (+/-0.007) for {'activation': 'tanh', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.97 (+/-0.009) for {'activation': 'tanh', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.97 (+/-0.007) for {'activation': 'tanh', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.97 (+/-0.006) for {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.971 (+/-0.005) for {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive'}
0.971 (+/-0.005) for {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rat

In [896]:
val_prediction = cv.predict(X_val)

In [897]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_val, val_prediction))

Accuracy score for Testing Dataset =  0.9757914338919925


In [898]:
test_prediction = cv.predict(X_test)

In [899]:
print('Accuracy score for Testing Dataset = ', accuracy_score(y_test, test_prediction))

Accuracy score for Testing Dataset =  0.9714640198511166


In [776]:
ind = data_norm.index
cali_data['r'][ind]

108      10
109      10
110      10
111      10
112      10
         ..
70868    10
70869    10
70870    10
70871    10
71298    10
Name: rebate, Length: 8056, dtype: int64

In [790]:
data['solar_system_count'][ind].value_counts()

0.0       253
12.0      111
11.0      107
10.0      107
4.0       106
         ... 
529.0       1
490.0       1
1116.0      1
596.0       1
524.0       1
Name: solar_system_count, Length: 541, dtype: int64

In [789]:
data_norm.columns

Index(['average_household_income', 'gini_index', 'per_capita_income',
       'population_density', 'education_less_than_high_school_rate',
       'education_high_school_graduate_rate', 'education_college_rate',
       'education_bachelor_rate', 'education_master_rate',
       'education_professional_school_rate', 'education_doctoral_rate',
       'race_white_rate', 'race_black_africa_rate', 'race_indian_alaska_rate',
       'race_asian_rate', 'race_islander_rate', 'race_other_rate',
       'race_two_more_rate', 'poverty_family_below_poverty_level_rate',
       'heating_fuel_gas_rate', 'heating_fuel_electricity_rate',
       'heating_fuel_fuel_oil_kerosene_rate', 'heating_fuel_coal_coke_rate',
       'heating_fuel_solar_rate', 'heating_fuel_other_rate',
       'heating_fuel_none_rate', 'household_count', 'average_household_size',
       'housing_unit_count', 'housing_unit_occupied_count',
       'housing_unit_median_value', 'housing_unit_median_gross_rent',
       'age_18_24_rate', 'age